In [31]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

In [32]:
tf.__version__

'2.11.0'

In [33]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [34]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [35]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [36]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [37]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

In [38]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(25 * 20, activation='sigmoid'))
    model.add(layers.Reshape((25, 20)))
    return model
    

    # model = Sequential([
    #     layers.Input(shape=(latent_dim,)),
    #     layers.Dense(128, activation='relu'),
    #     layers.Dense(256, activation='relu'),
    #     layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
    #     layers.Reshape(output_shape),
    #     layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
    #     layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    # ])

    # return model

In [39]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator()

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(1, 25, 20), dtype=float32, numpy=
array([[[0.6183278 , 0.5159866 , 0.56070536, 0.48719716, 0.36398685,
         0.46603346, 0.462894  , 0.63001984, 0.3109427 , 0.47952646,
         0.4812577 , 0.68320346, 0.494835  , 0.4279682 , 0.5214799 ,
         0.48720098, 0.61891735, 0.6444756 , 0.4186665 , 0.56531405],
        [0.63043535, 0.48973018, 0.46949843, 0.61755186, 0.34962735,
         0.5789172 , 0.38743177, 0.52229524, 0.6086862 , 0.552657  ,
         0.6451335 , 0.44995406, 0.37592936, 0.4265271 , 0.47425592,
         0.38870603, 0.4174372 , 0.45291737, 0.4855664 , 0.5406489 ],
        [0.41758332, 0.4631627 , 0.7293058 , 0.52312624, 0.48558998,
         0.609188  , 0.5070556 , 0.44890413, 0.3314407 , 0.5724512 ,
         0.3756082 , 0.32442918, 0.62455094, 0.57630205, 0.6044108 ,
         0.42950892, 0.6108189 , 0.40798673, 0.5896743 , 0.4006242 ],
        [0.4874911 , 0.43888655, 0.49013484, 0.5223715 , 0.5621511 ,
         0.4906344 , 0.42476758, 0.5465541 , 0.

In [40]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [41]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 112ms/step


array([[0.30780783, 0.6921922 ],
       [0.17504199, 0.824958  ],
       [0.31832418, 0.68167585],
       [0.18360284, 0.81639713],
       [0.19149974, 0.80850023]], dtype=float32)

In [42]:
noise = tf.random.normal([1000, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

over50 = 0
over25 = 0
over40 = 0
total = 0
for key, output in enumerate(prediction):
    if output[1] > 0.25:
        over25 += 1
    if output[1] > 0.4:
        over40 += 1
    if output[1] > 0.5:
        over50 += 1
    total += 1

print(over25 / total)
print(over40 / total)
print(over50 / total)

32/32 [==============================] - 0s 4ms/step
[[9.9999881e-01 1.2514822e-06]
 [9.9999964e-01 3.7884419e-07]
 [9.9999928e-01 7.3201142e-07]
 ...
 [9.9999940e-01 6.5050284e-07]
 [9.9999964e-01 3.9253493e-07]
 [9.9999952e-01 4.6214348e-07]]
0.0
0.0
0.0


In [43]:
def discriminator_loss(real_output, fake_output):
    ones_second_column = tf.ones_like(real_output[:, 1:2])
    zeros_first_column = tf.zeros_like(real_output[:, :1])
    custom_matrix_real = tf.concat([zeros_first_column, ones_second_column], axis=1)
    ones_second_column = tf.ones_like(fake_output[:, 1:2])
    zeros_first_column = tf.zeros_like(fake_output[:, :1])
    custom_matrix_fake = tf.concat([ones_second_column, zeros_first_column], axis=1)

    real_loss = tf.keras.losses.BinaryCrossentropy()(custom_matrix_real, real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(custom_matrix_fake, fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

discriminator_loss(discriminator.predict([X1[:5,:,:]]), prediction)

1/1 [==============================] - 0s 24ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=0.2717897>

In [44]:
def generator_loss(fake_output):
    ones_second_column = tf.ones_like(fake_output[:, 1:2])
    zeros_first_column = tf.zeros_like(fake_output[:, :1])
    custom_matrix = tf.concat([zeros_first_column, ones_second_column], axis=1)
    return tf.keras.losses.BinaryCrossentropy()(custom_matrix, fake_output)

generator_loss(prediction)
# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

<tf.Tensor: shape=(), dtype=float32, numpy=13.90133>

In [45]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [46]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 76ms/step
[[9.9999726e-01 2.7330827e-06]
 [9.9999952e-01 4.3494552e-07]
 [9.9999928e-01 7.1344823e-07]
 [9.9999952e-01 5.2333223e-07]
 [9.9999952e-01 4.3110580e-07]]
[[0.30780783 0.6921922 ]
 [0.17504199 0.824958  ]
 [0.31832418 0.68167585]
 [0.18360284 0.81639713]
 [0.19149974 0.80850023]]
tf.Tensor(13.986166, shape=(), dtype=float32)
tf.Tensor(0.27178952, shape=(), dtype=float32)


In [47]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [48]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 500
noise_dim = 100

In [49]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print(gen_loss)
    print(disc_loss)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(epochs):
  for epoch in range(epochs):
    dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


In [50]:
train(EPOCHS)

Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
Time for epoch 1 is 4.874107837677002 sec
Time for epoch 2 is 0.5278592109680176 sec
Time for epoch 3 is 0.46424317359924316 sec
Time for epoch 4 is 0.34880709648132324 sec
Time for epoch 5 is 0.4910550117492676 sec
Time for epoch 6 is 0.3827323913574219 sec
Time for epoch 7 is 0.48107385635375977 sec
Time for epoch 8 is 0.39923524856567383 sec
Time for epoch 9 is 0.631192684173584 sec
Time for epoch 10 is 0.3398001194000244 sec
Time for epoch 11 is 0.4551985263824463 sec
Time for epoch 12 is 0.366527795791626 sec
Time for epoch 13 is 0.5780384540557861 sec
Time for epoch 14 is 0.7999856472015381 sec
Time for epoch 15 is 0.940794229507446

In [61]:
noise = tf.random.normal([1000, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

over50 = 0
over25 = 0
over40 = 0
total = 0
for key, output in enumerate(prediction):
    if output[1] > 0.25:
        over25 += 1
    if output[1] > 0.4:
        over40 += 1
    if output[1] > 0.5:
        over50 += 1
    total += 1

print(over25 / total)
print(over40 / total)
print(over50 / total)

32/32 [==============================] - 0s 4ms/step
[[0.5454503  0.45454973]
 [0.47031873 0.52968127]
 [0.54570436 0.45429558]
 ...
 [0.49318314 0.50681686]
 [0.5340632  0.46593684]
 [0.49426886 0.50573117]]
1.0
0.993
0.139


In [52]:
rounds = 20
for i in range(rounds):
    noise = tf.random.normal([1000, 100])
    fake_seqs = generator(noise, training=False)

    prediction = discriminator.predict([fake_seqs[:,:15,:]])

    over50 = 0
    over25 = 0
    over40 = 0
    over45 = 0
    over5 = 0
    total = 0
    for key, output in enumerate(prediction):
        if output[1] > 0.25:
            over25 += 1
        if output[1] > 0.4:
            over40 += 1
        if output[1] > 0.45:
            over45 += 1
        if output[1] > 0.5:
            over50 += 1
        if output[1] > 0.05:
            over5 += 1
        total += 1

    print(f"round {i + 1}:a, over5%={over5 / total} over 25%={over25 / total}, over 40%={over40 / total}, over 45%={over45 / total}, over 50%={over50 / total}")

32/32 [==============================] - 0s 5ms/step
round 1:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.858, over 50%=0.027
32/32 [==============================] - 0s 2ms/step
round 2:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.852, over 50%=0.025
32/32 [==============================] - 0s 5ms/step
round 3:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.838, over 50%=0.029
32/32 [==============================] - 0s 7ms/step
round 4:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.859, over 50%=0.023
32/32 [==============================] - 0s 6ms/step
round 5:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.884, over 50%=0.029
32/32 [==============================] - 0s 6ms/step
round 6:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.86, over 50%=0.039
32/32 [==============================] - 0s 4ms/step
round 7:a, over5%=1.0 over 25%=1.0, over 40%=1.0, over 45%=0.853, over 50%=0.022
32/32 [==============================] - 0s 4ms/step
round 8:a,